ref. https://www.codewars.com/kata/55fd2d567d94ac3bc9000064/train/julia

Given the triangle of consecutive odd numbers:
```sh
                1
             3      5
           7     9     11
        13    15    17    19
     21    23    25    27     29
  31    33    35    37    39      41 
...
```

Calculate the row sums of this triangle from the row index (starting at index 1) e.g.:

`rowsumoddnumbers(1) # 1`  
`rowsumoddnumbers(2) # 3 + 5 = 8`

In [ ]:
# Can we build this triangle?

# n = 1, [1]
# n = 2, [3, 5]                [n+1, n+3] n is even
# n = 3, [7, 9, 11]            [n+4, n+6, n+8] n is odd
# n = 4, [13, 15, 17, 19]      [n+9, n+11, n+13, n+15] n is even 
# n = 5, [21, 23, 25, 27, 29]  [n+16, n+18, n+20, n+22, n+24]


# 1      2        3             4                 5                         6 
# 1, | 3, 5, | 7, 9, 11 | 13, 15, 17, 19 | 21, 23, 25, 27, 29 | 31, 33, 35, 37, 39, 41

# 1, 2 | 3, 4, 5 | 6, 7, 8, 9, 10, 11 | 12, 13, 14, 15, 16, 17, 18, 19 | 

In [1]:
using Test
using BenchmarkTools

In [2]:
function gen_odd_seq(start::Int64, n::Int64)
    """
    Calc. n-length sequence of successive odd number starting at start
    """
    iseven(start) && (start += 1)
    return n == 1 ? [1] : [start + 2 * j  for j in 1:n]
end

gen_odd_seq (generic function with 1 method)

In [3]:
@code_warntype gen_odd_seq(1, 1)

Variables
  #self#::Core.Compiler.Const(gen_odd_seq, false)
  start@_2::Int64
  n::Int64
  #3::getfield(Main, Symbol("##3#4"))
  start@_5::Union{}
  start@_6::Union{}
  start@_7::Union{Int64, Core.Box}

Body::Array{_A,1} where _A
1 ──       (start@_7 = start@_2)
│          (start@_7 = Core.Box(start@_7::Int64))
│          Core.NewvarNode(:(#3))
│    %4  = Core.isdefined(start@_7::Core.Box, :contents)::Bool
└───       goto #3 if not %4
2 ──       goto #4
3 ──       Core.NewvarNode(:(start@_5))
└───       start@_5
4 ┄─ %9  = Core.getfield(start@_7::Core.Box, :contents)::Any
│    %10 = Main.iseven(%9)::Union{Missing, Bool, getfield(Base, Symbol("##60#61")){_A} where _A}
└───       goto #9 if not %10
5 ── %12 = Core.isdefined(start@_7::Core.Box, :contents)::Bool
└───       goto #7 if not %12
6 ──       goto #8
7 ──       Core.NewvarNode(:(start@_6))
└───       start@_6
8 ┄─ %17 = Core.getfield(start@_7::Core.Box, :contents)::Any
│    %18 = (%17 + 1)::Any
│          Core.setfield!(start@_7:

In [4]:
a = gen_odd_seq(1, 1)
println(typeof(a))
@test a == [1]

Array{Int64,1}


Test Passed

In [5]:
a = gen_odd_seq(1, 2); println(a)
@test a == [3, 5]

[3, 5]


Test Passed

In [6]:
a = gen_odd_seq(5, 3); println(a)
@test a == [7, 9, 11]

[7, 9, 11]


Test Passed

In [13]:
a = gen_odd_seq(11, 4); println(a)
@test a == [13, 15, 17, 19]

[13, 15, 17, 19]


Test Passed

In [7]:
# t_s == type stable 
# using parametric type

function gen_odd_seq_ts(start::T, n::T) where T <: Integer 
    """
    Calc. n-length sequence of successive odd number starting at start
    """
    iseven(start) && (start += 1)
    a = fill(1, n)
    if n > 1
        a = fill(1, n)
        for jx in 1:n
            a[jx] = start + 2*jx
        end
    end
    return a
end

gen_odd_seq_ts (generic function with 1 method)

In [8]:
@code_warntype gen_odd_seq_ts(1, 10)

Variables
  #self#::Core.Compiler.Const(gen_odd_seq_ts, false)
  start@_2::Int64
  n::Int64
  a::Array{Int64,1}
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  jx::Int64
  start@_7::Int64

Body::Array{Int64,1}
1 ─       (start@_7 = start@_2)
│         Core.NewvarNode(:(a))
│         Core.NewvarNode(:(@_5))
│   %4  = Main.iseven(start@_7)::Bool
└──       goto #3 if not %4
2 ─       (start@_7 = start@_7 + 1)
└──       goto #4
3 ─       false
4 ┄       (a = Main.fill(1, n))
│   %10 = (n > 1)::Bool
└──       goto #8 if not %10
5 ─       (a = Main.fill(1, n))
│   %13 = (1:n)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_5 = Base.iterate(%13))
│   %15 = (@_5 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #8 if not %16
6 ┄ %18 = @_5::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (jx = Core.getfield(%18, 1))
│   %20 = Core.getfield(%18, 2)::Int64
│   %21 = start@_7::Int64
│   %22 = (2 * jx)::Int64
│   %23 = (%21 + %22

In [9]:
println(Base.return_types(gen_odd_seq_ts, (Int64,)))
println(Base.return_types(gen_odd_seq_ts, (Int64, Int64)))

Any[]
Any[Array{Int64,1}]


In [10]:
a = gen_odd_seq_ts(1, 1); println(a)
@test a == [1]

[1]


Test Passed

In [11]:
a = gen_odd_seq_ts(1, 2); println(a)
@test a == [3, 5]

[3, 5]


Test Passed

In [12]:
a = gen_odd_seq_ts(5, 3); println(a)
@test a == [7, 9, 11]

[7, 9, 11]


Test Passed

In [14]:
na = gen_odd_seq_ts(11, 4); println(na)
@test na == [13, 15, 17, 19]

[13, 15, 17, 19]


Test Passed

In [74]:
function init_fun(hsh, offset)
  if isempty(hsh)
    println("is empty")
    hsh[1] = 1 # no modification of hsh type
    # offset = 2
  else
    # hash was not empty, so compute offset as it was not set (because -1)    
    if offset == -1 
      offset = sort(collect(keys(hsh)))[end]
    end
  end
  return hsh, offset
end

function odd_num_triangle_builder_nt(n, hsh=Dict(), offset=-1, fun=gen_odd_seq_ts)
  println("offset: ", offset, " / hsh: ", hsh)
  @assert n >= 1

  eterm = 0
  hsh, offset = init_fun(hsh, offset) 
  offset == -1 && ((offset, eterm) = (2, -2))  

  ix_s, ix_e = offset, offset+n+eterm
  for ix in ix_s:ix_e
    start = hsh[ix-1][end]
    println("start = ", start, " hsh: ", hsh, " ix = ", ix)
    ary = fun(start, ix)
    println("start = ", start, " hsh: ", hsh, " ix = ", ix, " / ary = ", ary)
    hsh[ix] = ary
  end
  return hsh
end

odd_num_triangle_builder_nt (generic function with 4 methods)

In [16]:
@code_warntype odd_num_triangle_builder_nt(5) object of type Array{Int

Variables
  #self#::Core.Compiler.Const(odd_num_triangle_builder_nt, false)
  n::Int64

Body::Dict{Any,Any}
1 ─ %1 = Main.Dict()::Dict{Any,Any}
│   %2 = (#self#)(n, %1, -1, Main.gen_odd_seq_ts)::Dict{Any,Any}
└──      return %2


In [18]:
Base.return_types(odd_num_triangle_builder_nt, (Int64,))

1-element Array{Any,1}:
 Dict{Any,Any}

In [22]:
function print_sorted(hsh)
  for key in sort(collect(keys(hsh)))
    println("$key => $(hsh[key])")
  end
end

print_sorted (generic function with 1 method)

In [68]:
hash10 = odd_num_triangle_builder_nt(10)
print_sorted(hash10)

-1
is empty
start = 1 hsh: Dict{Any,Any}(1 => 1) ix = 2
start = 1 hsh: Dict{Any,Any}(1 => 1) ix = 2 / ary = [3, 5]
start = 5 hsh: Dict{Any,Any}(2 => [3, 5],1 => 1) ix = 3
start = 5 hsh: Dict{Any,Any}(2 => [3, 5],1 => 1) ix = 3 / ary = [7, 9, 11]
start = 11 hsh: Dict{Any,Any}(2 => [3, 5],3 => [7, 9, 11],1 => 1) ix = 4
start = 11 hsh: Dict{Any,Any}(2 => [3, 5],3 => [7, 9, 11],1 => 1) ix = 4 / ary = [13, 15, 17, 19]
start = 19 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],1 => 1) ix = 5
start = 19 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],1 => 1) ix = 5 / ary = [21, 23, 25, 27, 29]
start = 29 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],5 => [21, 23, 25, 27, 29],1 => 1) ix = 6
start = 29 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],5 => [21, 23, 25, 27, 29],1 => 1) ix = 6 / ary = [31, 33, 35, 37, 39, 41]
start = 41 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],5 => [21,

In [69]:
hash5 = odd_num_triangle_builder_nt(5)
print_sorted(hash5)

-1
is empty
start = 1 hsh: Dict{Any,Any}(1 => 1) ix = 2
start = 1 hsh: Dict{Any,Any}(1 => 1) ix = 2 / ary = [3, 5]
start = 5 hsh: Dict{Any,Any}(2 => [3, 5],1 => 1) ix = 3
start = 5 hsh: Dict{Any,Any}(2 => [3, 5],1 => 1) ix = 3 / ary = [7, 9, 11]
start = 11 hsh: Dict{Any,Any}(2 => [3, 5],3 => [7, 9, 11],1 => 1) ix = 4
start = 11 hsh: Dict{Any,Any}(2 => [3, 5],3 => [7, 9, 11],1 => 1) ix = 4 / ary = [13, 15, 17, 19]
start = 19 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],1 => 1) ix = 5
start = 19 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],1 => 1) ix = 5 / ary = [21, 23, 25, 27, 29]
1 => 1
2 => [3, 5]
3 => [7, 9, 11]
4 => [13, 15, 17, 19]
5 => [21, 23, 25, 27, 29]


In [70]:
hash2x5 = odd_num_triangle_builder_nt(5, hash5)
print_sorted(hash2x5)

-1
start = 19 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],5 => [21, 23, 25, 27, 29],1 => 1) ix = 5
start = 19 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],5 => [21, 23, 25, 27, 29],1 => 1) ix = 5 / ary = [21, 23, 25, 27, 29]
start = 29 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],5 => [21, 23, 25, 27, 29],1 => 1) ix = 6
start = 29 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],5 => [21, 23, 25, 27, 29],1 => 1) ix = 6 / ary = [31, 33, 35, 37, 39, 41]
start = 41 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],5 => [21, 23, 25, 27, 29],6 => [31, 33, 35, 37, 39, 41],1 => 1) ix = 7
start = 41 hsh: Dict{Any,Any}(4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11],5 => [21, 23, 25, 27, 29],6 => [31, 33, 35, 37, 39, 41],1 => 1) ix = 7 / ary = [43, 45, 47, 49, 51, 53, 55]
start = 55 hsh: Dict{Any,Any}(7 => [43, 45, 47, 49, 51, 53, 55],4 => [13, 15, 17, 19],2 => [3, 5],3 => [7, 9, 11

In [66]:
@test hash10 == hash2x5

Test Passed

In [75]:
hsh10 = Dict{Int64, Int64}()
# ofs = -1
hsh10 = odd_num_triangle_builder_nt(10, hsh10)
print_sorted(hsh10)

offset: -1 / hsh: Dict{Int64,Int64}()
is empty
start = 1 hsh: Dict(1 => 1) ix = 2
start = 1 hsh: Dict(1 => 1) ix = 2 / ary = [3, 5]


MethodError: MethodError: Cannot `convert` an object of type Array{Int64,1} to an object of type Int64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Integer}, !Matched::Ptr) where T<:Integer at pointer.jl:23
  ...

In [ ]:
one(Int64)

In [ ]:
hsh = odd_num_triangle_builder_nt(5) # 5 'rows'

print_sorted(hsh)

In [ ]:
hash = odd_num_triangle_builder_nt(5, hsh) # 5 more rows

print_sorted(hash)

In [ ]:
odd_num_triangle_builder_nt(0)

In [ ]:
function row_sum_odd_numbers(n, hsh=Dict()) 
  hsh = odd_num_triangle_builder_nt(n, hsh) # odd_num_triangle_builder_nt
  return sum(hsh[n]), hsh  
end

In [ ]:
@code_warntype row_sum_odd_numbers(5)

In [ ]:
using Printf

for i in 1:10
    res = @time row_sum_odd_numbers(i)[1]
    @printf("\t%d ==> %d\n", i, res)
end

In [ ]:
using Printf

hsh = Dict()
for i in 1:10
    res, hsh = @time row_sum_odd_numbers(i, hsh)
    @printf("\t%d ==> %d\n", i, res)
end

In [ ]:
using Test

@test true

In [ ]:
@test row_sum_odd_numbers(1)[1] == 1
@test row_sum_odd_numbers(2)[1] == 8

In [ ]:
hash = Dict()
val, hash = row_sum_odd_numbers(42)

@test val == 74088

In [ ]:
hash

#### Attempt at type

In [ ]:
function odd_num_triangle_builder(n::T, hsh::Dict{T, T}=Dict(), offset::T=-1, 
        fun=gen_odd_seq_ts) where T <: Integer
    
  # @assert n >= 1 # one(T)[1]
  e_term = 0
  hsh, offset = init_fun(hsh, offset) 
  offset == -1 && ((offset, e_term) = (2, -2))  

  ix_s, ix_e = offset, offset + n + e_term
  for ix in ix_s:ix_e
    start = hsh[ix-1][end]
    hsh[ix] = fun(start, ix)
  end

  return hsh
end

In [ ]:
@code_warntype odd_num_triangle_builder(5)